In [1]:
# Regular imports
import matplotlib.pyplot as plt
import torch
from torch import nn
import numpy as np 
device = "cuda" if torch.cuda.is_available() else "cpu"

# Import torch.nn.functional as F
import torch.nn.functional as F

In [2]:
# Define data path
data_path = "/kaggle/input/davidson1/labeled_data.csv"


In [3]:
# Load data
import pandas as pd
data = pd.read_csv(data_path)
df = pd.DataFrame(data)
df.columns = ['Unnamed: 0', 'count', 'hate_speech', 'offensive_language', 'neither',
              'classk', 'tweet']
df["classk"] = df["classk"].map({2:1, 1:0, 0:0})  # Convert labels to binary (hate vs not hate)
tweets = df.tweet
res = []


In [4]:
# Process tweets
for i in range(len(tweets)):
    splitt = tweets[i].split(":")
    if len(splitt) == 1:
        res.append(splitt[0])
    else:
        res.append(splitt[-1])

tweets = res

In [5]:

# Import tokenizer and BERT model
from transformers import AutoTokenizer, BertModel
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [6]:
# Define custom dataset class
class Custom_Dataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        
    def __getitem__(self, index: int) -> (torch.Tensor, int):
        X = self.data[index]
        y = self.labels[index]
        return (X, y)
    
    def __len__(self) -> int:
        return len(self.data)


In [7]:
# Split data into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tweets, list(df.classk), test_size=0.2, shuffle=True)

# Define train and test datasets and dataloaders
train_data = Custom_Dataset(data=X_train, labels=y_train)
test_data = Custom_Dataset(data=X_test, labels=y_test)
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=3, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=1, shuffle=False)

In [8]:
class Model0(nn.Module):
    def __init__(self, model, tokenizer):
        super().__init__()
        self.tokenizer = tokenizer
        self.model = model
        self.fc1 = nn.Linear(768, 256)  # Additional layer for fine-tuning
        self.fc2 = nn.Linear(256, 2)     # Output layer

    def forward(self, X):
        inputs = self.tokenizer(X, return_tensors="pt").to(device)
        outputs = self.model(**inputs)
        outputs = outputs.last_hidden_state[0][0].unsqueeze(1)

        
        # Additional layers for fine-tuning
        outputs = torch.relu(self.fc1(outputs.squeeze()))  # Remove extra dimension
        outputs = self.fc2(outputs)

        return outputs


In [9]:
# Instantiate the model
model0 = Model0(model, tokenizer)
model0.to(device)

# Define the optimizer
optimizer = torch.optim.Adam(params=model0.parameters(), lr=0.00002)

In [10]:
# Define the inter and intra space loss functions
def L_inter(spaces):
    means = [torch.mean(spaces[i]) for i in range(spaces.shape[0])]
    loss = torch.tensor(0.0, requires_grad=True)
    for k in range(len(means)):
        cur = torch.tensor(0.0, requires_grad=True)
        for l in range(len(means)):
            if l == k:
                continue
            cur = cur + (1 / (1 - ((means[k] * means[l]))))
        loss = loss + cur
    return loss

def L_intra(spaces):
    loss = torch.tensor(0.0, requires_grad=True)
    for k in range(len(spaces)):
        loss = loss + 1 / torch.var(spaces[k], dim=0)
    return torch.sum(loss) / loss.shape[0]

In [11]:
# Evaluate the model
correct = 0
total = len(y_test)

true = []
preds = []

with torch.no_grad():
    for i, batch in enumerate(test_loader):
#         if i == 100 :
#             break
        X, y = batch
        y_preds = model0(X)

        # Move y tensor to the same device as y_preds
        y = y.to(y_preds.device)

        y_predict = torch.argmax(y_preds)

        true.extend(y.cpu())  # Move true labels back to CPU
        preds.append(y_predict.cpu())  # Move predictions back to CPU

        cur = torch.eq(y_predict, y)

        correct += torch.sum(cur)

print(f"Test Accuracy: {correct/total}")


Test Accuracy: 0.3149082064628601


In [12]:
# Train the model
model0.train()
try:
    for _ in range(3):
        for i, batch in enumerate(train_loader):
#             if i==100:
#                 break
            if i % 2000 == 0:
                print(i)

            X, Y = batch
            loss = 0

            for X, y in zip(X, Y):
                y_preds = model0(X)
                loss += F.cross_entropy(y_preds.view(1, -1).to(device), torch.tensor([y]).to(device))

            loss /= len(Y)
#             loss += 0.7 * L_inter(torch.cat((hate_space, not_hate_space))) + 0.5 * L_intra(torch.stack([hate_space, not_hate_space]))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        

except KeyboardInterrupt:
    print("Training interrupted by the user.")

0
2000
4000
6000
0
2000
4000
6000
0
2000
4000
6000


In [13]:
# Evaluate the model
correct = 0
total = len(y_test)

true = []
preds = []

with torch.no_grad():
    for i, batch in enumerate(test_loader):
#         if i == 100 :
#             break
        X, y = batch
        y_preds = model0(X)

        # Move y tensor to the same device as y_preds
        y = y.to(y_preds.device)

        y_predict = torch.argmax(y_preds)

        true.extend(y.cpu())  # Move true labels back to CPU
        preds.append(y_predict.cpu())  # Move predictions back to CPU

        cur = torch.eq(y_predict, y)

        correct += torch.sum(cur)

print(f"Test Accuracy: {correct/total}")


Test Accuracy: 0.9227355122566223


In [14]:
# Define hate and not_hate words
hate_words = ['Moist', 'Cunt', 'Panties', 'Fuck', 'Hate', 'Nigger', 'Pussy', 'Ass', 
              'Motherfucker', 'Bitch', 'Damn']
not_hate_words = ['Love', 'Peace', 'Kindness', 'Happiness', 'Respect', 'Friendship',
                  'Appreciation', 'Hope', 'Encouragement', 'Support', 'Caring']

In [15]:
temp = []
for word in hate_words:
    inputs = tokenizer(word, return_tensors="pt").to(device)
    outputs = model(**inputs)
    outputs = torch.mean(outputs.last_hidden_state[0][1:], dim=0)
    temp.append(outputs.tolist())

hate_space = torch.tensor(temp).to(device).requires_grad_()

# Process not_hate words
temp = []
for word in not_hate_words:
    inputs = tokenizer(word, return_tensors="pt").to(device)
    outputs = model(**inputs)
    outputs = torch.mean(outputs.last_hidden_state[0][1:], dim=0)
    temp.append(outputs.tolist())

not_hate_space = torch.tensor(temp).to(device).requires_grad_()

In [16]:
import random
l=[[random.uniform(0,1)] for i in range(11)]
l

[[0.5720338785422512],
 [0.45928205225202356],
 [0.44936006997990596],
 [0.8108510437408157],
 [0.96053383712128],
 [0.7053073959196501],
 [0.5787197708418793],
 [0.5292252027504174],
 [0.10294530098888222],
 [0.16710931574901944],
 [0.6757277080630221]]

In [17]:
# next(iter(test_loader)) 
tensor_shape = (11,1)
hate_space_weights = torch.tensor(l).to(device).requires_grad_()
not_hate_space_weights = torch.tensor(l).to(device).requires_grad_()

print(hate_space_weights.device,not_hate_space_weights.is_leaf)

cuda:0 True


In [18]:
hate_space_weights.shape

torch.Size([11, 1])

In [19]:
hate_space_weights.is_leaf

True

In [20]:
import torch.nn.functional as F

class Model1(nn.Module):

    def __init__(self, model, tokenizer):
        super().__init__()
        self.tokenizer = tokenizer
        self.model = model

    def forward(self, X):
        inputs = self.tokenizer(X, return_tensors="pt").to(device)
        outputs = self.model(**inputs)
        outputs = outputs.last_hidden_state[0][1:len(outputs.last_hidden_state[0])].transpose(0, 1)#shape=>(768,no of tokens)
        col_norms = torch.norm(outputs, p=2, dim=0, keepdim=True)
        outputs = outputs / col_norms#makes every word rep unit vector 
        hate_att_scores=torch.matmul(hate_space/torch.norm(hate_space,p=2, dim=0, keepdim=True),outputs)#shape=>(11,v)
        not_hate_att_scores=torch.matmul(not_hate_space/torch.norm(not_hate_space,p=2, dim=0, keepdim=True),outputs)#shape=>(11,v)
        Psk_hate = torch.matmul(hate_att_scores.max(1)[0], hate_space_weights)#shape=>(1)
        Psk_not_hate = torch.matmul(not_hate_att_scores.max(1)[0], not_hate_space_weights)#shape=>(1)
        output_tensor = torch.cat((Psk_hate, Psk_not_hate), dim=0).requires_grad_()#
        return output_tensor

In [21]:
model1 = Model1(model, tokenizer)

In [22]:
def L_inter(spaces):
    """this function calculates inter space losses between spaces 
    forces the class spaces of different class to move apart
    """
    means = [torch.mean(spaces[i]) for i in range(spaces.shape[0])]
    
    loss = torch.tensor(0.0, requires_grad=True)
    
    for k in range(len(means)):
        cur = torch.tensor(0.0, requires_grad=True)
        for l in range(len(means)):
            if l == k:
                continue
            cur = cur + (1 / (1 - ((means[k] * means[l]))))
        
        loss = loss + cur
    
    return loss
            
            

def L_intra(spaces):
    """"this function calculates intra space loss 
forces the vector representations inside a class space to have high variability(varience)
"""
    loss=torch.tensor(0.0, requires_grad=True)
    for k in range(len(spaces)):
        loss=loss+1/torch.var(spaces[k], dim=0)
    
    return torch.sum(loss)/loss.shape[0]

In [23]:
print(hate_space,not_hate_space)

tensor([[ 0.2835,  0.3799, -0.0489,  ...,  0.4111, -0.7292,  0.0447],
        [-0.0774,  0.9263, -0.4947,  ..., -0.4127, -0.8346, -0.4496],
        [-0.2498,  0.7824, -0.5947,  ..., -0.2192, -1.0976, -0.6697],
        ...,
        [-0.1559,  0.8670, -0.3117,  ..., -0.4052, -1.0796, -0.5676],
        [-0.1798,  1.1351, -0.6005,  ..., -0.5616, -0.9880, -0.3972],
        [-0.1925,  0.6191, -0.6414,  ..., -0.7111, -1.2045, -0.4112]],
       device='cuda:0', requires_grad=True) tensor([[-0.1001,  0.0712, -0.2645,  ..., -0.5130, -0.6274,  0.3050],
        [-0.0923,  0.5072, -0.0767,  ...,  0.6526, -0.1610, -0.3062],
        [ 0.1175,  0.0554,  0.0812,  ...,  0.4107, -0.8082,  0.1769],
        ...,
        [ 0.1417,  0.1733, -0.1953,  ..., -0.4548, -0.9666,  0.2320],
        [-0.0031,  0.0814, -0.3989,  ..., -0.1432, -1.0147,  0.3762],
        [-0.0963,  0.2074,  0.0061,  ...,  0.3238, -0.5720,  0.0845]],
       device='cuda:0', requires_grad=True)


In [24]:
print(hate_space_weights,not_hate_space_weights)

tensor([[0.5720],
        [0.4593],
        [0.4494],
        [0.8109],
        [0.9605],
        [0.7053],
        [0.5787],
        [0.5292],
        [0.1029],
        [0.1671],
        [0.6757]], device='cuda:0', requires_grad=True) tensor([[0.5720],
        [0.4593],
        [0.4494],
        [0.8109],
        [0.9605],
        [0.7053],
        [0.5787],
        [0.5292],
        [0.1029],
        [0.1671],
        [0.6757]], device='cuda:0', requires_grad=True)


In [25]:
from sklearn.metrics import precision_score, recall_score


model1.to(device)

print(hate_space.is_leaf)
optimizer = torch.optim.Adam(params=[hate_space, not_hate_space, hate_space_weights, not_hate_space_weights], lr=0.0002)
model1.train()


curr=0.1
prev=0
pprev=0
for _ in range(5):
    for i, batch in enumerate(train_loader):
#         if i == 100:
#             break
        if i % 2000 == 0:
            print(i)

        x, Y = batch
        loss = 0

        for X, y in zip(x, Y):
            probs = model1(X)
#             print(y_preds)
            #print(f"printing y_preds={y_preds} shape={y_preds.shape}")

            loss+=F.cross_entropy(probs.view(1, -1).to(device), torch.tensor([y]).to(device))
            """explisit soft max is not used to convert to probs is cross 
            entrophy loss has inbuilt softmax  function adding softmax explisity
            will effect adversly by smoothing again on smoothed values will decrese 
            the weightage for class with high prob """
#             break
        loss /= len(x)
        loss += 1*L_inter(torch.cat((hate_space, not_hate_space))) + 1*L_intra(torch.stack([hate_space, not_hate_space]))
            
        optimizer.zero_grad()
        
        loss.backward()
        optimizer.step()
#         print(hate_space.grad,hate_space_weights.grad)
    correct = 0
    total = len(y_test)

    true = []
    preds = []

    with torch.inference_mode():
        for i, batch in enumerate(test_loader):
#             if i==100:
#                 break

            X, y = batch
            Psk_hate, Psk_not_hate = model1(X)

            y_predict = torch.argmax(torch.tensor([Psk_hate.mean().item(), Psk_not_hate.mean().item()]))

            true.extend(y)
            preds.append(y_predict)

            cur = torch.eq(y_predict, y)

            correct += torch.sum(cur)
    pprev=prev
    prev=curr
    print(f"Test Accuracy: {correct/total}")
    curr=correct/total
    print(precision_score(y_test, preds, average='macro'))
    print(recall_score(y_test, preds, average='macro'))
  

True
0
2000
4000
6000
Test Accuracy: 0.9267702102661133
0.8889044013645062
0.8385442106867239
0
2000
4000
6000
Test Accuracy: 0.9247528910636902
0.8873783736588614
0.8316270835252665
0
2000
4000
6000
Test Accuracy: 0.925963282585144
0.8858074268965358
0.8390091959897118
0
2000
4000
6000
Test Accuracy: 0.9245511293411255
0.8838300109107672
0.8353078550358344
0
2000
4000
6000
Test Accuracy: 0.9257615208625793
0.8885189024960458
0.8346103770813524


In [26]:
print(hate_space_weights,not_hate_space_weights)

tensor([[ 0.8066],
        [ 0.2633],
        [ 0.3671],
        [ 0.6730],
        [ 1.1703],
        [ 0.5149],
        [ 0.4067],
        [ 0.3660],
        [-0.1735],
        [ 0.0838],
        [ 0.7872]], device='cuda:0', requires_grad=True) tensor([[ 0.3909],
        [ 0.2029],
        [ 0.2682],
        [ 0.6173],
        [ 0.7399],
        [ 0.4918],
        [ 0.3752],
        [ 0.3130],
        [-0.0143],
        [-0.0584],
        [ 0.4951]], device='cuda:0', requires_grad=True)


In [27]:
print(hate_space,not_hate_space)

tensor([[ 1.6038, -1.5659,  0.6457,  ...,  1.2650,  0.3202,  1.5032],
        [ 3.5712,  4.5294,  1.3242,  ...,  1.1436,  1.1454, -3.9265],
        [-2.4002,  3.6308, -4.2167,  ...,  0.8800, -4.4099, -2.3879],
        ...,
        [-4.3117,  1.6197,  2.9765,  ...,  3.2159, -2.3051, -3.2518],
        [ 4.7351,  7.2255,  4.2718,  ...,  4.5184,  3.6693,  5.9725],
        [-1.6805, -1.6542, -2.1949,  ..., -2.0371, -2.2772,  1.3946]],
       device='cuda:0', requires_grad=True) tensor([[-2.4144, -1.6238,  1.1095,  ..., -1.6856,  0.4545,  1.7856],
        [-4.0897,  3.8164,  1.6111,  ...,  3.5819,  1.7304, -3.1038],
        [ 2.0563, -2.0605,  1.6224,  ...,  2.4679, -5.2721,  3.2191],
        ...,
        [ 3.3990, -4.2220,  4.5054,  ..., -4.8844, -3.1756,  3.8616],
        [ 3.8960, -4.0345, -4.2510,  ..., -4.7830, -3.3777,  3.6780],
        [-2.1914, -2.0175,  1.0585,  ...,  1.2781,  0.3304, -2.3755]],
       device='cuda:0', requires_grad=True)


In [28]:
def save_bert_model(model,tokenizer,params):
        output_dir = 'Saved/bert'
#         if(params['train_att']):
#             if(params['lambda_attn']>=1):
#                 params['lambda_attn']=int(params['lambda_attn'])

#             output_dir =  output_dir+str(params['supervised_layer_pos'])+'_'+str(params['num_supervised_heads'])+'_'+str(params['num_classes'])+'_'+str(params['lambda_attn'])+'/'
            
#         else:
#             output_dir=output_dir+'_'+str(params['num_classes'])+'/'
#         print(output_dir)
#         # Create output directory if needed
#         if not os.path.exists(output_dir):
#             os.makedirs(output_dir)

#         print("Saving model to %s" % output_dir)

        # Save a trained model, configuration and tokenizer using `save_pretrained()`.
        # They can then be reloaded using `from_pretrained()`
        model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
        model_to_save.save_pretrained(output_dir)
        tokenizer.save_pretrained(output_dir)

In [29]:
save_bert_model(model,tokenizer,model.parameters())

In [30]:
torch.save(hate_space, 'hatespace.pt')
torch.save(hate_space_weights, 'hatespace.pt_weights')
torch.save(not_hate_space, 'not_hatespace.pt')
torch.save(not_hate_space_weights, 'not_hatespace_weights.pt')